### Import Data

In [4]:
def unzipData(zipFile, outputLoc):
    import os
    import zipfile

    local_zip = zipFile

    zip_ref = zipfile.ZipFile(local_zip, 'r')

    zip_ref.extractall(outputLoc)
    zip_ref.close()

In [5]:
unzipData('datasetZip/dataset.zip', './data/')

In [6]:
!ls data

evaluation training   validation


In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths to your training, validation, and evaluation directories
train_dir = "data/training"
val_dir = "data/validation"
eval_dir = "data/evaluation"

# Define image size and batch size
image_size = (224, 224)  # Adjust as needed
batch_size = 32

# Create ImageDataGenerator instances for training, validation, and evaluation sets
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
eval_datagen = ImageDataGenerator(rescale=1./255)

# Create generators for training, validation, and evaluation sets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (one-hot encoded)
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

eval_generator = eval_datagen.flow_from_directory(
    eval_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Optionally, print class indices
print("Class indices:")
print(train_generator.class_indices)



Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.
Found 3347 images belonging to 11 classes.
Class indices:
{'Bread': 0, 'Dairy product': 1, 'Dessert': 2, 'Egg': 3, 'Fried food': 4, 'Meat': 5, 'Noodles-Pasta': 6, 'Rice': 7, 'Seafood': 8, 'Soup': 9, 'Vegetable-Fruit': 10}


In [14]:
input_shape = (224,224,3)
num_classes = 11

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
def create_model(input_shape, num_classes):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten layer
    model.add(layers.Flatten())

    # Dense layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Optional dropout layer for regularization
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Specify the input shape based on your images
input_shape = (224, 224, 3)  # Adjust based on the actual size and number of color channels in your images

# Specify the number of classes (assuming a multi-class classification scenario)
num_classes = 11  # Number of categories (Bread, Dairy product, ..., Vegetable-Fruit)

# Create the model
model = create_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 26, 26, 128)      

In [16]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # Adjust the number of epochs as needed
    validation_data=val_generator
)


Epoch 1/10


2023-12-19 21:33:58.985009: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


309/309 [==============================] - ETA: 0s - loss: 2.2712 - accuracy: 0.1984

2023-12-19 21:34:23.573670: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


309/309 [==============================] - 34s 105ms/step - loss: 2.2712 - accuracy: 0.1984 - val_loss: 2.0373 - val_accuracy: 0.2528
Epoch 2/10
309/309 [==============================] - 31s 102ms/step - loss: 2.0906 - accuracy: 0.2741 - val_loss: 1.8443 - val_accuracy: 0.3510
Epoch 3/10
309/309 [==============================] - 32s 102ms/step - loss: 1.9859 - accuracy: 0.3244 - val_loss: 1.8811 - val_accuracy: 0.3233
Epoch 4/10
309/309 [==============================] - 32s 102ms/step - loss: 1.8946 - accuracy: 0.3615 - val_loss: 1.7912 - val_accuracy: 0.3845
Epoch 5/10
309/309 [==============================] - 32s 102ms/step - loss: 1.8091 - accuracy: 0.4148 - val_loss: 1.9947 - val_accuracy: 0.3592
Epoch 6/10
309/309 [==============================] - 32s 103ms/step - loss: 1.7352 - accuracy: 0.4620 - val_loss: 1.9923 - val_accuracy: 0.3723
Epoch 7/10
309/309 [==============================] - 32s 102ms/step - loss: 1.7798 - accuracy: 0.5003 - val_loss: 2.5117 - val_accuracy: 0.3